# Example
#### Notebook with examples of using different submodules of our KBQA system

In [1]:
from caches.wikidata_entity_to_label import WikidataEntityToLabel
from caches.genre import GENREWikidataEntityesCache
from caches.wikidata_shortest_path import WikidataShortestPathCache

# WikidataEntityToLabel

In [6]:
entity2label = WikidataEntityToLabel()
print(entity2label.get_label("Q90"))

Paris


# WikidataShortestPathCache

In [12]:
shortest_path = WikidataShortestPathCache()
shortest_path.get_shortest_path("Q142", "Q90")

sleep 60...


['http://www.wikidata.org/entity/Q142', 'http://www.wikidata.org/entity/Q90']

# mGENRE

Firstly, clone https://github.com/SergeyPetrakov/mGENRE_MEL and check **mGENRE_and_Uncertainty_Estimation.ipynb** with preparation GENRE model steps

After that, you will have mGENRE_MEL directory with all that you need for **GENREWikidataEntityesCache module**

In [6]:
import pickle
import torch
from mGENRE_MEL.GENRE.genre.trie import Trie, MarisaTrie
from mGENRE_MEL.GENRE.genre.fairseq_model import mGENRE

with open("./mGENRE_MEL/lang_title2wikidataID-normalized_with_redirect.pkl", "rb") as f:
    lang_title2wikidataID = pickle.load(f)

with open("./mGENRE_MEL/titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
    trie = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_mGENRE = mGENRE.from_pretrained(
    "./mGENRE_MEL/fairseq_multilingual_entity_disambiguation"
).eval()
model_mGENRE.to(device)
print("mGENRE loaded")

mGENRE loaded


In [10]:
entity2label = WikidataEntityToLabel()
wikidata_label2entity_vocab_en = {v: k for k, v in entity2label.cache.items()}

genre_entityes = GENREWikidataEntityesCache(
    model_mGENRE,
    wikidata_label2entity_vocab_en,
    trie,
    lang_title2wikidataID,
)
genre_entityes.sentences_batch_to_entities(
    [
        "[START] in what french city did antoine de févin die  [END]",
        "[START] What job does jamie hewlett have [END]",
    ]
)

[{'Q2856873': 'Antoine de Févin',
  'Q7742': 'Louis XIV of France',
  'Q185075': 'Saint-Germain-en-Laye'},
 {}]

# Subgraph Extraction

In [28]:
from caches.wikidata_subgraphs_retriever import SubgraphsRetriever
from caches.wikidata_entity_to_label import WikidataEntityToLabel
from caches.wikidata_shortest_path import WikidataShortestPathCache
import matplotlib.pyplot as plt
import os


# sample question: in what french city did antoine de févin die
E1 = "Q22686"  # Doland Trump
E2 = "Q242351"  # Ivanka Trump
E3 = "Q432473"  # Melania Trump
Es = [E1, E2, E3]

C = "Q881"  # USA
entity2label = WikidataEntityToLabel()
shortest_path = WikidataShortestPathCache()
subgraph_obj = SubgraphsRetriever(entity2label, shortest_path, edge_between_path=False)
subgraph = subgraph_obj.get_subgraph(Es, C)
ax = subgraph_obj.visualize_subgraph(subgraph, Es, C)

path = "visualization"
if not os.path.exists(path):
    os.makedirs(path)

plt.savefig(path + "/sub.svg")
print("Visualization can be viewed at visualization/subgraph.svg")

IndexError: tuple index out of range